In [12]:
import pandas as pd
import numpy as np
import csv
import scipy.sparse


input_file = "ml-20m/ratings.csv"
name_data = pd.read_csv("ml-20m/movies.csv")
name_data.index.name = 'index'
movie_to_index = dict((m,i) for i,m in zip(name_data.index, name_data.movieId))
data = pd.read_csv(input_file, nrows=20_000_000)


In [22]:
import pandas as pd
import numpy as np
import csv
import scipy.sparse

ratings = np.zeros(shape=(139000, 27278))
for d in data.itertuples():
    movie = movie_to_index[getattr(d, 'movieId')]
    user = int(getattr(d, 'userId'))
    rating = getattr(d, 'rating')
    ratings[user][movie] = rating

done
done


In [ ]:
scipy.sparse.save_npz("np_outfile", scipy.sparse.csr_matrix(ratings))
